In [2]:
!pip install geopandas rasterio folium matplotlib sentinelhub


In [1]:
from sentinelhub import SHConfig, DataCollection, SentinelHubRequest, bbox_to_dimensions, SentinelHubDownloadClient, MimeType, BBox, CRS, Geometry
import datetime
import time

from superSecret import jupyter_client_id, jupyter_secret

# Configure your Sentinel Hub account
config = SHConfig()
config.sh_client_id = jupyter_client_id
config.sh_client_secret = jupyter_secret


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from superSecret import kenya_client_id, secret, jupyter_client_id, jupyter_secret

from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Your client credentials
client_id = 'sh-f409cb6a-725d-469c-a88d-159bfee1f813' 
#client_id = jupyter_client_id[0]
client_secret = 'en7cPBKZ81BAUzofUekgivFBbtpRCCRD' #jupyter_secret

#client_id = str(jupyter_client_id)
#client_secret = str(jupyter_secret[0])

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret)

# All requests using this session will have an access token automatically added
#resp = oauth.get("...")
#print(resp.content)
def sentinelhub_compliance_hook(response):
    response.raise_for_status()
    return response

oauth.register_compliance_hook("access_token_response", sentinelhub_compliance_hook)

In [3]:
#token['expires_at']
token_expires_at = datetime.datetime.fromtimestamp(token['expires_at'])
print(token_expires_at)

2024-03-18 14:52:05.231494


In [38]:
# Define the time range for the NDVI time series and boundingBox

#start_date = datetime.datetime(2024,2,2)
#end_date = datetime.datetime(2024,2,27)
## WIZ
start_date = datetime.datetime(2024,1,29)
end_date = datetime.datetime(2024,1,29)


#step_days = 5
step_days = 1

#date_range = [start_date + datetime.timedelta(days=x) for x in range((end_date-start_date).days + 1, step_days)]
date_range = [start_date + datetime.timedelta(days=x) for x in range(0, (end_date - start_date).days + 1, step_days)]

boundingBox = [
#                    38.6587,
#                    3.0251,
#                    38.8612,
#                    3.1515
                     9.85,
                    51.3,
                     9.95,
                    51.4
                     
]



#with open(evalscript_filename, 'r') as file:
#    evalscript_content = file.read()

rgb_nir = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B03", "B04", "B08"],
    output: { bands: 4 },
  }
}

function evaluatePixel(sample) {
  return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02]
}
"""

ndvi = """
//VERSION=3
function setup() {
  return {
    input: ["B04", "B08"], // red and nir bands
    output: { bands: 1 },  // NDVI output
  }
}

function evaluatePixel(sample) {
  return [(sample.B08 - sample.B04)/(sample.B08 + sample.B04)]  // NDVI calculation
}
"""
def getData(scriptName, locationName):
    for timestamp in date_range:
        request = {
            "input": {
                "bounds": {
                    "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
                    "bbox": boundingBox,
                },
                "data": [
                    {
                        "type": "sentinel-2-l2a",
                        "dataFilter": {
                            "timeRange": {
                                "from": timestamp.strftime('%Y-%m-%dT00:00:00Z'),
                                "to": timestamp.strftime('%Y-%m-%dT23:59:59Z'),
                            }
                        },
                    }
                ],
            },
            "output": {
                "width": 2500,
                "height": 2500,
            },
            "evalscript": scriptName,
            
        }
        
        url = "https://sh.dataspace.copernicus.eu/api/v1/process"
        
        response = oauth.post(url, json=request, headers={"Accept": "image/tiff"})
        if response.status_code == 200:
            # Specify the local file path where you want to save each timestamp's NDVI data
            local_file_path = f"./output/{locationName}_{timestamp.strftime('%Y%m%d')}.tif"
        
            # Open the local file in binary write mode and write the response content
            with open(local_file_path, "wb") as file:
                file.write(response.content)
    
            print(f"{scriptName} data for {timestamp} saved successfully to: {local_file_path}")
        elif response.status_code == 429:
            # 429 status code indicates rate limiting, wait for a minute
            print("Rate limit exceeded. Waiting for one minute...")
            time.sleep(60)
            # Retry the same request
            response = oauth.post(url, json=request, headers={"Accept": "image/tiff"})
        else:
            print(f"Error: {response.status_code}, {response.text}")
    
        # Introduce a delay between requests
        time.sleep(60 / 50)  # 60 seconds / 50 requests per minute

In [39]:
#getData(rgb_nir,"wiz")
getData(ndvi,"wiz_ndvi")


//VERSION=3
function setup() {
  return {
    input: ["B04", "B08"], // red and nir bands
    output: { bands: 1 },  // NDVI output
  }
}

function evaluatePixel(sample) {
  return [(sample.B08 - sample.B04)/(sample.B08 + sample.B04)]  // NDVI calculation
}
 data for 2024-01-29 00:00:00 saved successfully to: ./output/wiz_ndvi_20240129.tif
